In [82]:
import os 
import json
from pprint import pprint


dataset_path = '/home/furkan/Desktop/Citation_Datasets/dblp-ref'
counter = 0
dict_ids = {}
id_citedby = {}
id_citation_count = {}

for file in sorted(os.listdir(dataset_path)):
    with open(dataset_path + '/' + file) as data_file:    
        for line in data_file:
            data = json.loads(line)
            try:
                if data['year'] >= 2000 and data['year'] <= 2010 and len(data.keys()) >= 7:
                    dict_ids[data['id']] = data
                    counter += 1
                    if 'references' in data.keys():
                        for ref in data['references']:
                            if ref in id_citedby.keys():
                                id_citedby[ref].append(data['id'])
                                id_citation_count[ref] += 1
                            else:
                                id_citedby[ref] = [data['id']]
                                id_citation_count[ref] = 1 
            except:
                print('No year info')

In [83]:
common_ids = list(set(dict_ids.keys()) & set(id_citation_count.keys()))

In [84]:
print(len(common_ids)) #number of papers both published and get cited 2000-2010
print(len(id_citation_count.keys())) #number of papers cited  2000-2010
print(len(dict_ids.keys())) #number of papers published  2000-2010
print(counter)

628956
934856
1063957
1063957


In [86]:
#Looking at commmon_ids, filter with total number citations:
over_avg_5 = 0
avg_citation_id = {}
for key in common_ids:
    avg_citation_per_year = float(id_citation_count[key]/(2011-int(dict_ids[key]['year'])))
    if  avg_citation_per_year >= 5:
        over_avg_5 += 1
        avg_citation_id[key] = avg_citation_per_year
        
print(over_avg_5)

30685


In [ ]:
#2010>x>2000 and All keys available and avg_citation_per_year >= 10
    over_10 = 135527   
    over_20 = 59219
    over_50 = 15947
    over_100 = 5163

    over_avg_5 = 30685 **(using this)
    over_avg_10 = 9125 



#2010>x>2000 and All keys available
628956
934856
1063957
1063957

#2010>x>2000
718266
954290
1292991
1292991

# x>2000
1644214
1922721
2620722
2620722

In [ ]:
## Now raw data is ready. Next steps:
# 1.Split dataset yearly, for example papers from 2000-2010. 
# 2.Filter them with the following:
##       *All the keys available!!
##       *Avg citations per year > 5 OR total citation per/of author(s) > 150. (1st option is better for now)
# 3.Construct the graph (edge if they both cite same paper(s)) and find out their yearly
## citations for the following years (2010-2017)

In [87]:
print(dict_ids[list(dict_ids.keys())[0]])

{'title': 'On the Existence of Rainbow 4-Term Arithmetic Progressions', 'references': ['160bb414-f728-4e60-b142-2617a9fe657c', '37d5f030-3bf6-4af8-98ba-8e2b934c8fc9', 'b7ab15f2-1e86-4e3c-ac76-99dfb80dcbda'], 'venue': 'Graphs and Combinatorics', 'year': 2007, 'authors': ['David Conlon', 'Veselin Jungic', 'Radoš Radoičić'], 'abstract': 'For infinitely many natural numbers n, we construct 4-colorings of [n] \xa0=\xa0 {1, 2, ..., n}, with equinumerous color classes, that contain no 4-term arithmetic progression whose elements are colored in distinct colors. This result solves an open problem of Jungic et al. (Comb Probab Comput 12:599–620, 2003) Axenovich and Fon-der-Flaass (Electron J Comb 11:R1, 2004).', 'id': 'fa164463-ab8a-4e51-8dec-b552d6fc1ee1'}


In [97]:
## Find out edges, by looking at common references
edge_ids = {}
number_of_edges = 0
keys = avg_citation_id.keys()
print(len(keys))
for key in keys:
    for ref in dict_ids[key]['references']:
        for cited in id_citedby[ref]:
            if cited in keys and cited != key:
                if key in edge_ids.keys():
                    edge_ids[key].append(cited)
                else:
                    edge_ids[key] = [cited]
            #common_refs = list(set(dict_ids[key1]['references']) & set(dict_ids[key2]['references']))
                
print(len(edge_ids.keys()))

30685
30203


In [129]:
## Now construct the graph!! (only ids from avg_citation_id)
import networkx as nx

G=nx.Graph()

G.add_nodes_from(avg_citation_id.keys())

for key in edge_ids.keys():
    for neighbor in edge_ids[key]:
        G.add_edge(key,neighbor)

In [123]:
#Write the graph
nx.info(G)
nx.write_gpickle(G,"CITATION_GRAPH.gpickle")

In [1]:
import networkx as nx
G = nx.read_gpickle("CITATION_GRAPH.gpickle")
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 30685\nNumber of edges: 2765946\nAverage degree: 180.2800'

In [2]:
nx.draw(G)
#plt.savefig("graph.png", dpi = 1000)
plt.savefig('graph.pdf')

NameError: name 'plt' is not defined